In [ ]:
```python
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
ticker = input("Enter Stock Ticker: ")
start = input("Enter start date (YYYY-MM-DD): ")
end = input("Enter end date (YYYY-MM-DD): ")

def price_data(ticker, start, end):
    data = yf.download(
        ticker,
        start=start,
        end=end,
        progress=False
    )
    data["mv1"] = data["Close"].rolling(50).mean()
    data["mv2"] = data["Close"].rolling(20).mean()
    
    data["Signal"] = 0


    data.loc[
    (data["mv2"] > data["mv1"]) &
    (data["mv2"].shift(1) <= data["mv1"].shift(1)),
    "Signal"
]  = 1


    data.loc[
    (data["mv2"] < data["mv1"]) &
    (data["mv2"].shift(1) >= data["mv1"].shift(1)),
    "Signal"
] = -1
    data["Position"] = data["Signal"].replace(0, method="ffill")
    data["Position"] = data["Position"].clip(lower=0)
    data["Position"] = data["Position"].shift(1)
    data["Market Return"] = data["Close"].pct_change()
    data["Strategy Return"] = data["Market Return"] * data["Position"]
    
    return data
     
prices = price_data(ticker, start, end)
prices["Market Cumulative"] = (1 + prices["Market Return"]).cumprod()
prices["Strategy Cumulative"] = (1 + prices["Strategy Return"]).cumprod()
print(
    prices[["Close", "mv2", "mv1", "Signal", "Position", "Strategy Return", "Market Return"]]
    .tail(100)
)
Strategy_Mean = prices["Strategy Return"].mean()
Strategy_SD = prices["Strategy Return"].std()
Market_Mean = prices["Market Return"].mean()
Market_SD = prices["Market Return"].std()
Strategy_Sharpe = (Strategy_Mean / Strategy_SD) * np.sqrt(250)
Market_Sharpe = (Market_Mean / Market_SD) * np.sqrt(250)
print("Strategy Sharpe is:", Strategy_Sharpe)
print("Market Sharpe is:", Market_Sharpe)
plt.figure(figsize=(10, 6))
plt.plot(prices.index, prices["Market Cumulative"], label="Buy & Hold")
plt.plot(prices.index, prices["Strategy Cumulative"], label="MA Crossover Strategy")
plt.legend()
plt.title("Cumulative Returns")
plt.xlabel("Date")
plt.ylabel("Growth")
plt.show()
```
